In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

In [2]:
ds = xr.open_dataset('../Data/ERA5/ERA5_LowRes_Monthly_t2m.nc')
df_temp = ds.to_dataframe().reset_index()
df_temp = df_temp[(df_temp['latitude'] > 25) & (df_temp['latitude'] < 50) & (df_temp['longitude'] > -126) & (df_temp['longitude'] < -67)]
df_temp

,longitude,latitude,time,t2m
8354880,-125.625,49.5,1979-01-01,269.944794
8354881,-125.625,49.5,1979-02-01,271.705597
8354882,-125.625,49.5,1979-03-01,274.698395
8354883,-125.625,49.5,1979-04-01,275.729492
8354884,-125.625,49.5,1979-05-01,277.936218
...,...,...,...,...
17393755,-67.125,25.5,2018-08-01,300.176147
17393756,-67.125,25.5,2018-09-01,300.306580
17393757,-67.125,25.5,2018-10-01,299.379486
17393758,-67.125,25.5,2018-11-01,298.447083


In [3]:
ds = xr.open_dataset('../Data/ERA5/ERA5_LowRes_Monthly_snow.nc')
df_snow = ds.to_dataframe().reset_index()
df_snow = df_snow[(df_snow['latitude'] > 25) & (df_snow['latitude'] < 50) & (df_snow['longitude'] > -126) & (df_snow['longitude'] < -67)]
df_snow

,longitude,latitude,time,rsn,sd,sf,tsn
8354880,-125.625,49.5,1979-01-01,273.608673,0.326400,0.004350,269.282623
8354881,-125.625,49.5,1979-02-01,288.498871,0.602750,0.015250,271.231476
8354882,-125.625,49.5,1979-03-01,390.177429,0.830727,0.001572,272.599091
8354883,-125.625,49.5,1979-04-01,407.716675,0.822639,0.003534,272.778595
8354884,-125.625,49.5,1979-05-01,421.009979,0.531488,0.000260,273.057251
...,...,...,...,...,...,...,...
17393755,-67.125,25.5,2018-08-01,99.999954,0.000000,0.000000,302.052368
17393756,-67.125,25.5,2018-09-01,99.999954,0.000000,0.000000,302.252380
17393757,-67.125,25.5,2018-10-01,99.999954,0.000000,0.000000,301.532684
17393758,-67.125,25.5,2018-11-01,99.999954,0.000000,0.000000,299.939392


In [4]:
ds = xr.open_dataset('../Data/ERA5/ERA5_LowRes_Monthly_clouds.nc')
df_clouds = ds.to_dataframe().reset_index()
df_clouds = df_clouds[(df_clouds['latitude'] > 25) & (df_clouds['latitude'] < 50) & (df_clouds['longitude'] > -126) & (df_clouds['longitude'] < -67)]
df_clouds

,longitude,latitude,time,hcc,lcc,mcc,tcc,tciw,tclw
8354880,-125.625,49.5,1979-01-01,0.440823,0.474433,0.372153,0.709801,0.034482,0.070613
8354881,-125.625,49.5,1979-02-01,0.544778,0.730487,0.617082,0.885454,0.090242,0.146817
8354882,-125.625,49.5,1979-03-01,0.382913,0.454138,0.311791,0.629916,0.027076,0.100386
8354883,-125.625,49.5,1979-04-01,0.294594,0.448492,0.366323,0.633457,0.035740,0.055119
8354884,-125.625,49.5,1979-05-01,0.458865,0.533533,0.352100,0.777584,0.031230,0.100293
...,...,...,...,...,...,...,...,...,...
17393755,-67.125,25.5,2018-08-01,0.239317,0.155754,0.101327,0.392233,0.009139,0.038346
17393756,-67.125,25.5,2018-09-01,0.283892,0.159965,0.096576,0.429421,0.015318,0.036943
17393757,-67.125,25.5,2018-10-01,0.162683,0.181054,0.131070,0.349689,0.011163,0.038284
17393758,-67.125,25.5,2018-11-01,0.223212,0.194772,0.140980,0.425819,0.007342,0.044384


In [5]:
ds = xr.open_dataset('../Data/ERA5/ERA5_LowRes_Monthly_tp.nc')
df_precip = ds.to_dataframe().reset_index()
df_precip = df_precip[(df_precip['latitude'] > 25) & (df_precip['latitude'] < 50) & (df_precip['longitude'] > -126) & (df_precip['longitude'] < -67)]
df_precip

,longitude,latitude,time,tp
8354880,-125.625,49.5,1979-01-01,0.005055
8354881,-125.625,49.5,1979-02-01,0.019228
8354882,-125.625,49.5,1979-03-01,0.005614
8354883,-125.625,49.5,1979-04-01,0.004564
8354884,-125.625,49.5,1979-05-01,0.003249
...,...,...,...,...
17393755,-67.125,25.5,2018-08-01,0.003133
17393756,-67.125,25.5,2018-09-01,0.002774
17393757,-67.125,25.5,2018-10-01,0.002825
17393758,-67.125,25.5,2018-11-01,0.002862


In [6]:
temp_snow = pd.merge(
    left = df_temp,
    right = df_snow,
    left_on= ['longitude', 'latitude', 'time'],
    right_on = ['longitude', 'latitude', 'time'],
    how = 'inner'
)
temp_snow_clouds = pd.merge(
    left = temp_snow,
    right = df_clouds,
    left_on= ['longitude', 'latitude', 'time'],
    right_on = ['longitude', 'latitude', 'time'],
    how = 'inner'
)

total = pd.merge(
    left = temp_snow_clouds,
    right = df_precip,
    left_on= ['longitude', 'latitude', 'time'],
    right_on = ['longitude', 'latitude', 'time'],
    how = 'inner'
)

In [7]:
total

,longitude,latitude,time,t2m,rsn,sd,sf,tsn,hcc,lcc,mcc,tcc,tciw,tclw,tp
0,-125.625,49.5,1979-01-01,269.944794,273.608673,0.326400,0.004350,269.282623,0.440823,0.474433,0.372153,0.709801,0.034482,0.070613,0.005055
1,-125.625,49.5,1979-02-01,271.705597,288.498871,0.602750,0.015250,271.231476,0.544778,0.730487,0.617082,0.885454,0.090242,0.146817,0.019228
2,-125.625,49.5,1979-03-01,274.698395,390.177429,0.830727,0.001572,272.599091,0.382913,0.454138,0.311791,0.629916,0.027076,0.100386,0.005614
3,-125.625,49.5,1979-04-01,275.729492,407.716675,0.822639,0.003534,272.778595,0.294594,0.448492,0.366323,0.633457,0.035740,0.055119,0.004564
4,-125.625,49.5,1979-05-01,277.936218,421.009979,0.531488,0.000260,273.057251,0.458865,0.533533,0.352100,0.777584,0.031230,0.100293,0.003249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251355,-67.125,25.5,2018-08-01,300.176147,99.999954,0.000000,0.000000,302.052368,0.239317,0.155754,0.101327,0.392233,0.009139,0.038346,0.003133
1251356,-67.125,25.5,2018-09-01,300.306580,99.999954,0.000000,0.000000,302.252380,0.283892,0.159965,0.096576,0.429421,0.015318,0.036943,0.002774
1251357,-67.125,25.5,2018-10-01,299.379486,99.999954,0.000000,0.000000,301.532684,0.162683,0.181054,0.131070,0.349689,0.011163,0.038284,0.002825
1251358,-67.125,25.5,2018-11-01,298.447083,99.999954,0.000000,0.000000,299.939392,0.223212,0.194772,0.140980,0.425819,0.007342,0.044384,0.002862


# Dataset Summary
Monthly climate data filtered for the United States. Includes temperature, snow depth/cover, precipitation, and cloud coverage.

# Source
https://fabienmaussion.info/climate_system/download.html